In [14]:
using JuMP, Gurobi, HiGHS

# Problema Completo: 3 horizontes

## Propuesta 1: modelo
Sirve para ver como funciona el sub problema anidado.

In [34]:
master = Model(Gurobi.Optimizer)

# IDEA: FUNCION QUE DEPENDE DE AFLUENTE

I = 3   # generadores termicos
T = 3   # periodos de tiempo

@variable(master, pT[1:I, 1:T] >= 0)    # Potencia activa de generador térmico i en tiempo t. Valor en p.u.
@variable(master, v[1:T] >= 0)          # Almacenamiento de agua
@variable(master, s[1:T] >= 0)          # Vertimiento de agua
@variable(master, pH[1:T])              # Potencia activa del generador hídrico (agua gastada)

demanda = [150, 150, 150]               # Demanda: cte
a = [50, 25, 25]                        # Afluentes: deterministico
cv = [50, 100, 150]                     # Costos variables
v0 = 100                                # Volumen inicial

## CONSTRAINTS

# Ax >= b
@constraint(master, Balance[t in 1:T], sum(pT[i,t] for i in 1:I) + pH[t] == demanda[t])     # demanda = generado
@constraint(master, PmaxT[i in 1:I, t in 1:T], pT[i,t] <= 50)                               # Pmax termico
@constraint(master, PmaxH[t in 1:T], pH[t] <= 150)                                          # Pmax hidrico
@constraint(master, MaxStorage[t in 1:T], v[t] <= 300)                                      # Almacenamiento V maximo
@constraint(master, VolumenInicial, v[1] == v0 + a[1] - s[1] - pH[1])                       # Volumen en t=1

# Ex + Fy >= g
@constraint(master, Acople[t in 2:T], v[t] == v[t-1] + a[t] - s[t] - pH[t])                 # Acople temporal de Almacenamiento
#@constraint(master, algo[t in 1:T], v[t] == 0)

@objective(master, Min, sum(cv[i]*pT[i,t] for i in 1:I, t in 1:T))

JuMP.optimize!(master)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 18 columns and 38 nonzeros
Model fingerprint: 0x640b6c54
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+02]
Presolve removed 18 rows and 7 columns
Presolve time: 0.00s
Presolved: 3 rows, 11 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.125000e+01   0.000000e+00      0s
       2    1.7500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.750000000e+04

User-callbac

### Imprimir algunos resultados

In [40]:
JuMP.value.(pH)

3-element Vector{Float64}:
  50.0
  50.0
 100.0

In [39]:
JuMP.value.(pT)

3×3 Matrix{Float64}:
 50.0  50.0  50.0
 50.0  50.0   0.0
  0.0   0.0   0.0

In [43]:
JuMP.value.(v)

3-element Vector{Float64}:
 100.0
  75.0
   0.0

## Propuesta 2: función que depende del estado anterior, la demanda y el afluente
* Se resuelve esta misma función varias veces.
* Para evaluar cada etapa se cambian los parámetros de entrada.
* Agregar un corte de Bender significa agregar una restricción en la función.
* Luego, para evaluar la siguiente etapa se eliminan los cortes.
* Originalmente, se resuelve un caso **determinístico** entre la etapa **1** y **2**.

In [131]:
function subProblema(x, demanda, afluente)
    v_prev = x
    a = afluente
    d = demanda
    cv = [50, 100, 150]
    # Voy a resolver el sub problema: despacho en un periodo
    # El problema está definido por x -> potencias despachadas -> volumen almacenado
    
    # x: volumen inicial del nuevo periodo

    sub = Model(Gurobi.Optimizer)

    ## Variables: sin componente temporal
    @variable(sub, pT[1:I] >= 0)    # Potencia activa de generador térmico i en tiempo t. Valor en p.u.
    @variable(sub, pH >= 0)         # Potencia activa del generador hídrico (agua gastada)  
    @variable(sub, s >= 0)          # Vertimiento de agua
    @variable(sub, v >= 0)          # Almacenamiento de agua

    @variable(sub, theta >= 0)      # theta

    #@variable(sub, v_prev >= 0)    # Almacenamiento fijo de etapa anterior

    ## Restricciones

    # Independientes    
    @constraint(sub, Balance, sum(pT[i] for i in 1:I) + pH == d)     # demanda = generado
    @constraint(sub, PmaxT[i in 1:I], pT[i] <= 50)                   # Pmax termico
    @constraint(sub, PmaxH, pH <= 150)                               # Pmax hidrico
    @constraint(sub, MaxStorage, v <= 300)                           # Almacenamiento V maximo

    # Dependientes de etapa anterior: Ax + By >= b
    @constraint(sub, AcopleTemp, v + s + pH == a + v_prev)           # Acople temporal de Almacenamiento
    
    #@constraint(sub, FixStorage, v_prev == v_prev_fijo)

    # solo v_prev va a la derecha (fijado de restriccion anterior)
    # Ax >= b - By*

    # Ax == b - By*      y: decisiones etapa anterior
    # v + s + pH == a + v_prev
    # [1 1 1]x == a + [1]y*

    # Corte: theta >= lambda(a+x*):    $/MWh

    # https://jump.dev/JuMP.jl/stable/tutorials/algorithms/benders_decomposition/
    ## CORTES BENDERS:    # theta >= theta(v*) + - pi([1])(v-v*)
    @constraint(sub, b1, theta >= 11250-150*(v-0))  
    @constraint(sub, b2, theta >= 2500-50*(v-75)) # por cada incremento de v respecto al punto anterior (v=75) puedo reducir el costo de la siguiente etapa en 50: reduzcamoslo
    @constraint(sub, b3, theta >= 5000-100*(v-50))

    @objective(sub, Min, theta + sum(cv[i]*pT[i] for i in 1:I))

    JuMP.optimize!(sub)
    
    # TENGO QUE SACAR EL CMg de una unidad mas de lamacenamiento
    lambda_v = dual(AcopleTemp)    # Este agrega el corte de Bender
    v_final = value(v)
    costo_total = objective_value(sub)
    costo_futuro = value(theta)
    return Dict("dual" => lambda_v, "vol" => v_final, "VO" => costo_total, "theta" => costo_futuro)
end

subProblema (generic function with 1 method)

- Etapa 1: v = 100, a = 50
- Etapa 2: v = ? , a = 25

subProblema(x, demanda, afluente)

In [132]:
sol = subProblema(100, 150, 50)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 7 columns and 18 nonzeros
Model fingerprint: 0xe885ed57
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+04]
Presolve removed 7 rows and 3 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.367188e+03   0.000000e+00      0s
       1    7.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  7.500000000e+03

User-callback c

Dict{String, Float64} with 4 entries:
  "vol"   => 50.0
  "theta" => 5000.0
  "dual"  => -100.0
  "VO"    => 7500.0

In [130]:
sol2 = subProblema(50, 150, 25)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 7 columns and 12 nonzeros
Model fingerprint: 0x718a440e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 3e+02]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.875000e+01   0.000000e+00      0s
       1    5.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.000000000e+03

User-callback cal

Dict{String, Float64} with 4 entries:
  "vol"   => 0.0
  "theta" => 0.0
  "dual"  => -100.0
  "VO"    => 5000.0

In [80]:
println(" ")
println("******************")
println("Variable dual volumen:", sol["dual"])
println("Volumen final etapa:", sol["vol"])
println("Costo total:", sol["VO"])
println("Costo siguiente etapa:", sol["theta"])

 
******************
Variable dual volumen:0.0
Volumen final etapa:0.0
Costo total:0.0
Costo siguiente etapa:0.0
